### Fera formidável 4.2 - Stop right now, thank you very much 

#### Enunciado:
Objetivo: implemente uma estratégia de Parada Antecipada (Early Stopping) no processo de treino da rede neural feita em Python puro ou no processo de treino da rede neural feita em PyTorch

Comentário: esta não é para resolver com o módulo lightning

### Introdução

Essa fera 4.2 consiste em implementar a estratégia de Early Stopping em uma rede neural. A estratégia Early Stopping consiste em parar o treinamento da rede neural assim que o seu desempenho para dados de validação começa a diminuir, evitando o overfitting.

A imagem abaixo demonstra como funciona essa estratégia. (Referência 3)

<img alt="Alt text" title="Early Stopping" src="../Imagens/Early_stopping.png">

A ideia para implementar o Early Stopping (que nesse caso será feito em uma rede neural treinada com pytorch) é treinar a rede e a cada época avaliar o modelo com o conjunto de validação, se o modelo melhorar o treinamento continua e se não melhorar após um determinado número de épocas o código retorna o modelo que teve o menor erro para os dados de validação.

### Importações

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import time

### Definindo classes

In [2]:
class MLP(nn.Module):
    def __init__(self, num_dados_entrada, neuronios_c1, neuronios_c2, num_targets):
        super().__init__()
        
        self.camadas = nn.Sequential(
            nn.Linear(num_dados_entrada, neuronios_c1),
            nn.Sigmoid(),
            nn.Linear(neuronios_c1, neuronios_c2),
            nn.Sigmoid(),
            nn.Linear(neuronios_c2, num_targets),
        )
        
    def forward(self, x):
        x = self.camadas(x)
        return x

### Definindo o dataset

In [3]:
df = sns.load_dataset("iris")
df

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,virginica
146,6.3,2.5,5.0,1.9,virginica
147,6.5,3.0,5.2,2.0,virginica
148,6.2,3.4,5.4,2.3,virginica


### Dividindo os dados em treino e teste

In [4]:
TAMANHO_TESTE = 0.1

indices = df.index
indices_treino, indices_teste = train_test_split(
    indices, test_size=TAMANHO_TESTE
)

df_treino = df.loc[indices_treino]
df_teste = df.loc[indices_teste]

In [5]:
df_treino

,sepal_length,sepal_width,petal_length,petal_width,species
50,7.0,3.2,4.7,1.4,versicolor
39,5.1,3.4,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa
8,4.4,2.9,1.4,0.2,setosa
79,5.7,2.6,3.5,1.0,versicolor
...,...,...,...,...,...
142,5.8,2.7,5.1,1.9,virginica
16,5.4,3.9,1.3,0.4,setosa
65,6.7,3.1,4.4,1.4,versicolor
55,5.7,2.8,4.5,1.3,versicolor


In [6]:
# P/ treino da rede
x_treino = df_treino["petal_length"].values
y_verdadeiro_treino = df_treino["petal_width"].values

x_treino = torch.tensor(x_treino)
x_treino = x_treino.view(-1, 1)
y_verdadeiro_treino = torch.tensor(y_verdadeiro_treino)
y_verdadeiro_treino = y_verdadeiro_treino.view(-1, 1)

# P/ validação da rede
x_validacao = df_teste["petal_length"].values
y_verdadeiro_validacao = df_teste["petal_width"].values

x_validacao = torch.tensor(x_validacao)
x_validacao = x_validacao.view(-1, 1)
y_verdadeiro_validacao = torch.tensor(y_verdadeiro_validacao)
y_verdadeiro_validacao = y_verdadeiro_validacao.view(-1,1)

In [7]:
x_treino = x_treino.float()
y_verdadeiro_treino = y_verdadeiro_treino.float()

x_validacao = x_validacao.float()
y_verdadeiro_validacao = y_verdadeiro_validacao.float()

### Criando a MLP

In [8]:
NUM_DADOS_DE_ENTRADA = 1
NUM_DADOS_DE_SAIDA = 1
NEURONIOS_C1 = 3
NEURONIOS_C2 = 2

In [9]:
minha_mlp = MLP(
    NUM_DADOS_DE_ENTRADA, NEURONIOS_C1, NEURONIOS_C2, NUM_DADOS_DE_SAIDA
)

In [10]:
y_prev = minha_mlp(x_treino)

TAXA_DE_APRENDIZADO = 0.005

otimizador = optim.SGD(minha_mlp.parameters(), lr=TAXA_DE_APRENDIZADO)
fn_perda = nn.MSELoss()

### Implementando o Early Stopping

In [11]:
NUMERO_EPOCAS_SEM_MELHORA = 15
MAXIMO_EPOCAS = 100000 # Critério de parada máximo para o while

melhor_rmse = float('inf') # Float infinito para que o próximo RMSE seja obrigatoriamente menor que ele
cont = 0
cont_maximo = 0
epoca_melhor_modelo = 0

inicio = time.time()

while cont < NUMERO_EPOCAS_SEM_MELHORA and cont_maximo < MAXIMO_EPOCAS:
    
    # Treino
    minha_mlp.train()

    y_pred = minha_mlp(x_treino)
    otimizador.zero_grad()
    loss = fn_perda(y_verdadeiro_treino, y_pred)
    loss.backward()
    otimizador.step()

    # Teste
    minha_mlp.eval()

    with torch.no_grad():
        y_pred = minha_mlp(x_validacao)
        
    RMSE = mean_squared_error(y_verdadeiro_validacao, y_pred, squared=False)
    
    if RMSE < melhor_rmse:
        melhor_rmse = RMSE
        cont = 0
        epoca_melhor_modelo = cont_maximo 
        melhor_desempenho = torch.save(minha_mlp.state_dict(), 'checkpoint.pt') # Salva o modelo atual como o melhor modelo (Referência 1)
        
    else:
        cont += 1

    cont_maximo += 1
    print(f"Época {cont_maximo} | RMSE: {RMSE:.4f}")
    
fim = time.time()
tempo = fim - inicio

print()
print(f"O tempo para rodar o código foi de {tempo:.2f} segundos, o que é equivalente a {tempo/60:.2f} minutos")

minha_mlp.load_state_dict(torch.load('checkpoint.pt')) # Recarrega a mlp como o melhor modelo alcançado pela rede neural

Época 1 | RMSE: 2.0688
Época 2 | RMSE: 2.0458
Época 3 | RMSE: 2.0231
Época 4 | RMSE: 2.0008
Época 5 | RMSE: 1.9788
Época 6 | RMSE: 1.9571
Época 7 | RMSE: 1.9358
Época 8 | RMSE: 1.9148
Época 9 | RMSE: 1.8941
Época 10 | RMSE: 1.8737
Época 11 | RMSE: 1.8537
Época 12 | RMSE: 1.8339
Época 13 | RMSE: 1.8145
Época 14 | RMSE: 1.7953
Época 15 | RMSE: 1.7764
Época 16 | RMSE: 1.7579
Época 17 | RMSE: 1.7396
Época 18 | RMSE: 1.7215
Época 19 | RMSE: 1.7038
Época 20 | RMSE: 1.6863
Época 21 | RMSE: 1.6691
Época 22 | RMSE: 1.6522
Época 23 | RMSE: 1.6355
Época 24 | RMSE: 1.6191
Época 25 | RMSE: 1.6029
Época 26 | RMSE: 1.5870
Época 27 | RMSE: 1.5713
Época 28 | RMSE: 1.5558
Época 29 | RMSE: 1.5406
Época 30 | RMSE: 1.5256
Época 31 | RMSE: 1.5109
Época 32 | RMSE: 1.4964
Época 33 | RMSE: 1.4821
Época 34 | RMSE: 1.4681
Época 35 | RMSE: 1.4542
Época 36 | RMSE: 1.4406
Época 37 | RMSE: 1.4272
Época 38 | RMSE: 1.4140
Época 39 | RMSE: 1.4010
Época 40 | RMSE: 1.3882
Época 41 | RMSE: 1.3756
Época 42 | RMSE: 1.3632
É

<All keys matched successfully>

### Resultado obtido com o Early Stopping

In [12]:
print(f"O melhor modelo foi obtido na época {epoca_melhor_modelo}, com RMSE de {melhor_rmse:.4f}")

O melhor modelo foi obtido na época 7786, com RMSE de 0.2471


### Conclusão

A estratégia de Early Stopping se mostrou poderosa para o que propõe. Treinar o modelo com ela impede que a rede neural fique overfitada para os dados de treino ao mesmo tempo que melhora ao máximo a previsão para novos dados.

### Referências

1: What is a state_dict in PyTorch — PyTorch Tutorials 2.4.0+cu121 documentation. Disponível em: <https://pytorch.org/tutorials/recipes/recipes/what_is_state_dict.html>.

2: KASHYAP, P. Early Stopping in Deep Learning: A Simple Guide to Prevent Overfitting. Disponível em: <https://medium.com/@piyushkashyap045/early-stopping-in-deep-learning-a-simple-guide-to-prevent-overfitting-1073f56b493e>.

3: Papers with Code - Early Stopping Explained. Disponível em: <https://paperswithcode.com/method/early-stopping>.